In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

print("Veriyi Anlama ve Hazırlama (Data Understanding & Preparation")
print("Veri setini yükleyin ve ilk gözlemleri yapın (head(), info(), describe() vb.).")
print("Veri setindeki sütunların (değişkenlerin) ne anlama geldiğini araştırın ve açıklayın.")
print("\n")

print(df.head())

print ("-"*100)

print(df.info())

print ("-"*100)

print ("Veri tiplerini kontrol edin ve anlamsal olarak hatalı olanları (örneğin sayısal olması gereken bir sütunun object olması) düzeltin.")
print("\n")
print(df.describe())

print ("."*100)
print ("."*100)

# 2) NaN sayıları ve oranları
print("Eksik verileri (NaN değerler) tespit edin. Bu eksik verilerle nasıl başa çıktığınızı (siliyor musunuz, dolduruyor musunuz, neden?) gerekçeleriyle açıklayın.")
print("\n")

na_counts = df.isna().sum().sort_values(ascending=False)
na_ratio = (df.isna().mean()*100).sort_values(ascending=False)
print("NaN sayıları:\n", na_counts)
print("\nNaN oranları (%):\n", na_ratio)

print ("."*100)
print ("."*100)

# 3) ∞ (inf) değerleri kontrol (özellikle 'Rate' için şüphe var)
num_cols = df.select_dtypes(include=[np.number]).columns
inf_mask = np.isinf(df[num_cols])
inf_counts = inf_mask.sum().sort_values(ascending=False)
print("\n∞ (inf) sayıları:\n", inf_counts)

print ("-"*100)

# 1-3
# 1) ±inf -> NaN
df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)

# 2) Sütun bazında NaN oranları
na_ratio = df.isna().mean()*100

# 3) Medyan ile doldurma (oranı düşük olan sayısal sütunlara)
fill_cols = [c for c in num_cols if na_ratio[c] > 0 and na_ratio[c] <= 5]
for c in fill_cols:
    df[c] = df[c].fillna(df[c].median())

# 4) Oranı yüksek sütunlar için karar (rapora yazmak için listeleyelim)
high_na_cols = [c for c in num_cols if na_ratio[c] > 5]
print("NaN oranı yüksek sütunlar:", high_na_cols)

# (Bu veri dosyasında genelde yalnızca 'Rate'te sorun olur; medyan ile doldurmak çoğu zaman yeterlidir.)
# Eğer sadece 'Rate' problemli ise, direkt medyan ile dolduralım:
if 'Rate' in df.columns:
    df['Rate'] = df['Rate'].fillna(df['Rate'].median())

# 5) Satır bazında çok eksik olanları sil (genelde gerekmez; güvenlik için eşik koyalım)
row_na_ratio = df.isna().mean(axis=1)*100
df = df.loc[row_na_ratio <= 20].copy()

# 6) Son kontrol
print("Temizlik sonrası toplam NaN:", int(df.isna().sum().sum()))
print("Rate örnek istatistikleri:\n", df['Rate'].describe())

print ("-"*100)

df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv") # digerlerine de bak!
print(df.dtypes)

print ("-"*100)
print ("-"*100)

# 1-5
print("Varsa aykırı değerleri (outliers) tespit edin. Bu değerleri nasıl ele aldığınızı ve neden bu yöntemi seçtiğinizi belirtin.")
print("\n")
import seaborn as sns
import matplotlib.pyplot as plt

# Örnek: Rate sütunu için boxplot
cols = ["Rate", "IAT", "AVG", "Time_To_Live"]

for col in cols:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=df[col], color="lightblue")
    plt.title(f"Boxplot of {col}")
    plt.show()

print ("-/"*100)

# 1-5
cols = ["Rate", "IAT", "AVG", "Time_To_Live"]

for col in cols:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=df[col], color="lightblue")
    plt.title(f"Boxplot of {col}")
    plt.show()
    
    # Aykırı değer analizi
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR

    outliers = df[(df[col] < lower) | (df[col] > upper)]
    print(f"{col} → Aykırı değer sayısı: {len(outliers)}")

print ("-"*100)
print ("-"*100)

# 2-1
print("Tek Değişkenli Analiz (Univariate Analysis)")
print("Sayısal değişkenlerin dağılımını histogram ve kutu grafiği (box plot) gibi grafiklerle görselleştirin. Her bir grafik için gözlemlerinizi (dağılımın çarpıklığı, merkezi eğilim vb.) yazın.")
print("\n")
import matplotlib.pyplot as plt
import seaborn as sns

num_vars = ["Rate", "AVG", "IAT"]

for col in num_vars:
    plt.figure(figsize=(12,4))
    
    # Histogram
    plt.subplot(1,2,1)
    sns.histplot(df[col], bins=50, kde=True)
    plt.title(f"{col} - Histogram")
    
    # Boxplot
    plt.subplot(1,2,2)
    sns.boxplot(x=df[col])
    plt.title(f"{col} - Boxplot")
    
    plt.show()

print ("-"*100)
print ("-"*100)

# 2-2
print("Kategorik değişkenlerin frekanslarını bar grafikleri ile görselleştirin.")
print("\n")

# --------------------------------------------------------------------------------------------------------------------------------------
# Protocol Type dağılımı
plt.figure(figsize=(8,5))
sns.countplot(x="Protocol Type", data=df)
plt.title("Protocol Type - Frekans Dağılımı")
plt.show()

print ("-"*100)

plt.figure(figsize=(8,5))
sns.countplot(x="ICMP", data=df)
plt.title("ICMP - Frekans Dağılımı")
plt.show()

print ("-"*100)

plt.figure(figsize=(8,5))
sns.countplot(x="TCP", data=df)
plt.title("TCP - Frekans Dağılımı")
plt.show()

print ("-"*100)

plt.figure(figsize=(8,5))
sns.countplot(x="UDP", data=df)
plt.title("UDP - Frekans Dağılımı")
plt.show()

print ("-"*100)

plt.figure(figsize=(8,5))
sns.countplot(x="ARP", data=df)
plt.title("ARP - Frekans Dağılımı")
plt.show()

print ("-"*100)
print ("-"*100)

# 3-1
# 3-2
print("Çok Değişkenli Analiz (Bivariate/Multivariate Analysis)")
print("Değişkenler arasındaki ilişkileri keşfedin.")
print("Korelasyon Analizi: Sayısal değişkenler arasındaki korelasyon matrisini oluşturun ve bir ısı haritası (heatmap) ile görselleştirin. Güçlü pozitif veya negatif ilişkileri yorumlayın.")
print("\n")

import seaborn as sns
import matplotlib.pyplot as plt

# Korelasyon matrisi
corr = df.corr()

plt.figure(figsize=(12,8))
sns.heatmap(corr, cmap="coolwarm", annot=False, linewidths=0.5)
plt.title("Korelasyon Matrisi - Heatmap")
plt.show()

print ("-"*100)

# 3-3
print("Saçılım Grafikleri (Scatter Plots): Özellikle ilginç bulduğunuz değişken çiftleri arasındaki ilişkiyi saçılım grafikleriyle inceleyin.")
print("\n")
sns.scatterplot(x="Rate", y="Time_To_Live", data=df, alpha=0.5)
plt.title("Rate vs TTL")
plt.show()

sns.scatterplot(x="Rate", y="Tot size", data=df, alpha=0.5)
plt.title("Rate vs Tot Size")
plt.show()

sns.scatterplot(x="AVG", y="Tot size", data=df, alpha=0.5)
plt.title("AVG vs Tot size")
plt.show()

print ("-"*100)
print ("-"*100)

# 3-4
print("Karşılaştırmalı Analiz: Kategorik ve sayısal değişkenleri bir arada analiz edin.")
print("\n")

files = {
    "Benign": "WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv",
    "ARP_Spoofing": "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv",
    "MQTT-DoS-Connect_Flood": "WiFI_and_MQTT/attacks/CSV/test/MQTT-DoS-Connect_Flood_test.pcap.csv"
}

frames = []
for label, path in files.items():
    temp = pd.read_csv(path)
    temp["Label"] = label  # saldırı türünü kategori olarak ekle
    # inf -> NaN
    num_cols = temp.select_dtypes(include=[np.number]).columns
    temp[num_cols] = temp[num_cols].replace([np.inf, -np.inf], np.nan)
    # NaN doldurma (ör: Rate için medyan)
    if "Rate" in temp.columns:
        temp["Rate"] = temp["Rate"].fillna(temp["Rate"].median())
    frames.append(temp)

# Tümünü tek DataFrame'e birleştir
df_all = pd.concat(frames, ignore_index=True)

print(df_all["Label"].value_counts())
plt.figure(figsize=(10,6))
sns.boxplot(x="Label", y="Rate", data=df_all)  # df_all = birden fazla dosyayı birleştirdiğimiz tablo
plt.xticks(rotation=45)
plt.title("Saldırı Türüne Göre Rate Dağılımı")
plt.show()

print ("-"*100)



In [ ]:
import pandas as pd, numpy as np
import seaborn as sns, matplotlib.pyplot as plt

pd.set_option("display.max_columns", None)
files = {
    "Benign": "WiFI_and_MQTT/attacks/CSV/test/Benign_test.pcap.csv",
    "ARP_Spoofing": "WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv",
    "MQTT-DoS-Connect_Flood": "WiFI_and_MQTT/attacks/CSV/test/MQTT-DoS-Connect_Flood_test.pcap.csv",
}

dfs = []
for label, path in files.items():
    df = pd.read_csv(path)
    df["Label"] = label
    # sayısal sütunlarda ±inf -> NaN
    num_cols = df.select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].replace([np.inf, -np.inf], np.nan)
    # Rate özelinde medyanla doldurma (sağa çarpık dağılımlarda sağlam)
    if "Rate" in df.columns:
        df["Rate"] = df["Rate"].fillna(df["Rate"].median())
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

# Protocol Type'ı hem int hem isimli göstermek için eşleştirme (raporda anlaşılır olsun)
proto_map = {1: "ICMP", 6: "TCP", 17: "UDP"}
df_all["Protocol Type Name"] = df_all["Protocol Type"].map(proto_map).fillna(df_all["Protocol Type"].astype(str))

######

def iqr_outlier_count(s: pd.Series):
    Q1, Q3 = s.quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5*IQR, Q3 + 1.5*IQR
    return ((s < lower) | (s > upper)).sum()

######
#Soru 1
print("SORU 1")

fig, axes = plt.subplots(1,2, figsize=(12,4))
sns.boxplot(data=df_all, x="Label", y="Rate", ax=axes[0])
axes[0].set_title("Rate dağılımı (Label)")
sns.boxplot(data=df_all, x="Label", y="IAT", ax=axes[1])
axes[1].set_title("IAT dağılımı (Label)")
plt.tight_layout(); plt.show()

print("-"*100)

summary_q1 = df_all.groupby("Label")[["Rate","IAT"]].agg(["median","mean","std"]).round(2)
summary_q1

print("-"*100)

In [ ]:
# Protocol Type (isimli) dağılımı (Label'a göre %)
print("SORU 2")
proto_dist = (df_all
    .pivot_table(index="Label", columns="Protocol Type Name", values="Number", aggfunc="count", fill_value=0))
proto_dist_pct = (proto_dist.div(proto_dist.sum(axis=1), axis=0)*100).round(1)
display(proto_dist_pct)

# ARP/TCP/UDP bayrak kolonları (0/1) Label bazında toplam
flag_cols = [c for c in ["ARP","TCP","UDP","ICMP","HTTP","HTTPS","DNS"] if c in df_all.columns]
flag_sum = df_all.groupby("Label")[flag_cols].sum().astype(int)
display(flag_sum)

# Görselleştirme: stacked bar (Protocol Type Name yüzdeleri)
proto_dist_pct.plot(kind="bar", stacked=True, figsize=(8,4))
plt.title("Protocol Type % dağılımı (Label)")
plt.ylabel("%"); plt.legend(bbox_to_anchor=(1.02,1), loc="upper left"); plt.tight_layout(); plt.show()


In [ ]:
#
print ("SORU 3")

sns.boxplot(data=df_all, x="Label", y="AVG")
plt.title("AVG (ortalama paket boyutu) — Label karşılaştırması"); plt.show()

# Outlier sayıları (AVG, Rate için) Label bazında
for col in ["AVG","Rate"]:
    counts = df_all.groupby("Label")[col].apply(iqr_outlier_count).astype(int)
    print(f"{col} outlier sayıları (IQR):"); display(counts)


In [ ]:
print("Soru 4")

plt.figure(figsize=(6,5))
sns.scatterplot(data=df_all, x="IAT", y="AVG", hue="Label", alpha=0.5)
plt.title("IAT vs AVG (saldırı sınıflarına göre)"); plt.legend(bbox_to_anchor=(1.02,1))
plt.xscale("log");  # IAT çok çarpık ise log eksen okunurluğu artırır
plt.tight_layout(); plt.show()


In [ ]:
# Protocol Type - INT64
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ARP Spoofing dosyasını oku
df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

# Orijinal int64 hali
plt.figure(figsize=(6,4))
sns.countplot(x="Protocol Type", data=df, palette="Blues")
plt.title("Protocol Type (int64 hali)")
plt.show()

# Kategoriye çevirelim (mapping)
proto_map = {1: "ICMP", 6: "TCP", 17: "UDP", 0: "ARP"}
df["Protocol Type_cat"] = df["Protocol Type"].map(proto_map).fillna(df["Protocol Type"].astype(str))

plt.figure(figsize=(6,4))
sns.countplot(x="Protocol Type_cat", data=df, palette="Greens")
plt.title("Protocol Type (kategori hali)")
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(6,3))
sns.boxplot(x=df["IAT"], color="lightblue")
plt.title("Boxplot of IAT (0–20 aralığı)")
plt.xlim(0.02, 0.09)   # sadece 0–20 arasını göster
plt.show()


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

print("NaN değerler:\n", df.isna().sum())
print("\nInf değerler:\n", np.isinf(df).sum())

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv("WiFI_and_MQTT/attacks/CSV/test/ARP_Spoofing_test.pcap.csv")

df[~np.isfinite(df).all(axis=1)]